In [1]:
import lyricsgenius as genius
import json
import re
import os
import pandas as pd
from wordcloud import WordCloud
import gensim
from gensim import corpora, models
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [31]:
#Connect Genius.com API credentials and select artist
geniusCreds = "Jw2VCOyiGXdDET_ZIY0ZllMd0UkoYOEkUB_1dBYMvd4yZx3fJk8DnJsZ0MW71qft"
api = genius.Genius(geniusCreds)
artist_name = "migos"

In [ ]:
artist = api.search_artist(artist_name)

Searching for songs by migos...

Changing artist name to 'Migos'
Song 1: "Bad and Boujee"
Song 2: "Versace (Remix)"
Song 3: "T-Shirt"
Song 4: "Slippery"
Song 5: "Walk It Talk It"
Song 6: "Stir Fry"
Song 7: "Kelly Price"
Song 8: "Narcos"
Song 9: "Get Right Witcha"
Song 10: "Fight Night"
Song 11: "Handsome and Wealthy"
Song 12: "Notice Me"
Song 13: "Call Casting"
Song 14: "Deadz"
Song 15: "Cocoon"
Song 16: "Hannah Montana"
Song 17: "Freak No More"
Song 18: "BBO (Bad Bitches Only)"
Song 19: "Bando"
Song 20: "China Town"
Song 21: "What the Price"
Song 22: "Gang Gang"
Song 23: "Culture"
Song 24: "Pipe It Up"
Song 25: "White Sand"
Song 26: "Wishy Washy"
Song 27: "Look at My Dab"
Song 28: "Brown Paper Bag"
Song 29: "Commando"
Song 30: "Cross the Country"
Song 31: "Versace"
Song 32: "Dab of Ranch"
Song 33: "Antidope"
Song 34: "All Ass"
Song 35: "Big on Big"
Song 36: "Supastars"
Song 37: "One Time"
Song 38: "Out Yo Way"


In [28]:
import os
os.getcwd()
artist.save_lyrics()

Wrote `Lyrics_LilUziVert.json`


In [ ]:
with open('Lyrics_TravisScott.json') as json_data:
    data = json.load(json_data)

In [ ]:
songs, albums, lyrics = [], [], []
for song in data.get('songs'):
    songs.append(song.get('full_title'))
    if song.get('album') is not None:
        albums.append(song.get('album').get('name'))
    else: albums.append('Single')
    song_lyrics = song.get('lyrics')
    #remove identifiers like chorus, verse, etc
    song_lyrics = re.sub(r'[\(\[].*?[\)\]]', '', song_lyrics)
    #remove empty lines
    song_lyrics = os.linesep.join([s for s in song_lyrics.splitlines() if s]) 
    lyrics.append(song_lyrics)

In [ ]:
corpus = pd.DataFrame({'song':songs, 'album':albums, 'lyrics':lyrics})
corpus.to_csv('travis_scott.csv', header=True, index=False)

In [ ]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatizer.lemmatize(token))
    return result

In [ ]:
corpus['contents'] = corpus['lyrics'].map(preprocess)
corpus['lyrics'] = [' '.join(map(str, l)) for l in corpus['contents']]
lda_songs = corpus[['song', 'contents']]

In [ ]:
corpus

In [ ]:
import matplotlib.pyplot as plt
import random
def custom_colours(*args, **kwargs):
    return "#D61347"
cloud = WordCloud(background_color = "#F5EFE0", color_func = custom_colours, max_words = 250, contour_width = 5, width = 750, height = 500, random_state = 5)
cloud.generate("".join(list(corpus["lyrics"].values)))
#plt.imshow(cloud.recolor(color_func=random_color_func, random_state=3),interpolation="bilinear")
cloud.to_image()

In [ ]:
file = lda_songs.values   ## this cell is for putting all the files in a dictionary shape.
file_dictionary = {}
for i in file:
    file_dictionary[i[0]] = i[1]

In [ ]:
processed_files = []
for key in file_dictionary:
    processed_files.append(file_dictionary[key])
dictionary = gensim.corpora.Dictionary(processed_files)

In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_files]
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [ ]:
topic_number = 10
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=topic_number, id2word=dictionary, passes=2, workers=2)

In [ ]:
for idx, topic in lda_model.show_topics(num_topics=10, num_words=5, log=False, formatted=True): ## change num_words to see number of words you want to see in a topic
    print('Topic: {} \nWords: {}'.format(idx+1, topic))

In [ ]:
pd.read_csv('travis_scott.csv')